In [ ]:
Задача
По имеющимся данным портала eclass.cc построить content-based рекомендации по образовательным курсам.
Запрещено использовать библиотеки pandas, sklearn и аналогичные.

Описание данных
Имеются следующие данные на вход:

набор данных о всех курсах. Датасет можно взять с HDFS по адресу: /labs/slaba02/DO_record_per_line.json
id курсов, для которых надо дать рекомендации (указаны в Личном кабинете).


Результат
Для каждого id курса из личного кабинета необходимо дать топ-10 наиболее похожих на него курсов. 
Рекомендованные курсы должны быть того же языка, что и курс, для которого строится рекомендация.

Выходной формат — json — должен иметь следующую структуру:

{
  "123": [
    5372,
    16663,
    23114,
    13079,
    13084,
    ...
  ],
  "456": [
    ...
  ],
  "789": [
    ...
  ],
  "123456": [
    ...
  ],
  "456789": [
    ...
  ],
  "987654": [
    ...
  ]
}
Ключи json — это id курсов, для которых строится рекомендация.
Для каждого такого ключа в качестве значения задается массив рекомендованных курсов, состоящий из их id,
отсортированных по убыванию метрики. При равенстве значений метрики курсов сортируются лексикографически по названию.

Также возможна очень редкая ситуация (в основном с русскоязычными курсами), когда в рекомендацию попадут два дубликата одного
курса, но с разными id. Таких дубликатов очень мало относительно числа курсов, но все равно рекомендуется их сортировать
в следующей последовательности: по метрике (убывание) => по названию (лексикографически по возрастанию) => по возрастанию id.

Также вы можете найти так называемый submission-файл по следующему пути на
мастер-ноде: /share/submission-files/slaba02/lab02.json. 
обладает правильной структурой и форматом, но неправильными значениями. Идея в том, что он проходит необходимые требования 
чекера именно по структуре, и вам не придется пытаться понять, в чем дело, а также почему чекер не принимает ваш файл.

Советы
Для подбора рекомендаций следует использовать меру TFIDF, а в качестве метрики для ранжирования — косинус угла между 
TFIDF-векторами для разных курсов

Что такое TFIDF? TF — это term frequency: по сути, сколько раз слово встречается в этом документе. Если мы сделаем такой
    word count по каждому документу, то получим вектор, который как-то характеризует этот документ.

мама - 2
мыла - 1
раму - 1
лапу - 1
роза - 2
упала - 3
Если мы сравним вектора, рассчитав дистанцую между ними, то получим вывод – насколько похожи эти тексты.
Назовем этот подход наивным. Этот подход наивен, потому что мы как бы присваиваем одинаковый вес каждому слову,
которое у нас есть в тексте. А что если мы попробуем как-то повысить значимость тех слов, которые часто встречаются 
только в этом тексте? Для этого мы посчитаем DF – document frequency: по сути, число документов, в которых есть вхождение 
    этого слова. Мы хотим "штрафовать" слово за частое появление в документах, поэтому делаем инверсию этой величины – 
    буква I в TFIDF. Теперь для каждого слова мы будем считать TF и делить на IDF. Так мы получим другой вектор для нашего 
    документа. Он может быть более правильным для наших задач.

TFIDF нужно считать для описаний курсов (desc). При извлечении слов из описания словом считаем то, что состоит из латинских
или кириллических букв или цифр, знаки препинания и прочие символы не учитываются.

Для поиска слов можно использовать такой код на Python (может быть проблема с распознаванием юникода).
import re
regex = re.compile(u'[\w\d]{2,}', re.U)
regex.findall(string.lower())
 
Сам TFIDF реализован в Spark, писать с нуля вычисления не требуется. При вычислении TF с помощью HashingTF использовалось 
число фичей: 10000. То есть:
 
 tf = HashingTF(10000).

In [1]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 2 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.7
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


In [2]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as f
from pyspark import Row
import json
import re
from pyspark.ml.feature import HashingTF, IDF

conf = SparkConf()

spark.conf.set("spark.sql.crossJoin.enabled", True)

spark = (SparkSession
         .builder
         .config(conf=conf)
         .appName("test")
         .getOrCreate())

In [3]:
df = spark.read.json("hdfs:///labs/slaba02/DO_record_per_line.json")

In [4]:
def tokenization(string):
    regex = re.compile(u'[\w\d]{2,}', re.U)
    return regex.findall(string.lower())
tokenization = f.udf(tokenization, ArrayType(StringType()))
df = df.withColumn("words", tokenization(df.desc))
df = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=10000).transform(df)
df = IDF(inputCol="rawFeatures", outputCol="features").fit(df).transform(df)

In [6]:
# id courses to make recommendations
a = [[23126, u'en', u'Compass - powerful SASS library that makes your life easier'], 
     [21617, u'en', u'Preparing for the AP* Computer Science A Exam \u2014 Part 2'], 
     [16627, u'es', u'Aprende Excel: Nivel Intermedio by Alfonso Rinsche'], 
     [11556, u'es', u'Aprendizaje Colaborativo by UNID Universidad Interamericana para el Desarrollo'], 
     [16704, u'ru', u'\u041f\u0440\u043e\u0433\u0440\u0430\u043c\u043c\u0438\u0440\u043e\u0432\u0430\u043d\u0438\u0435 \u043d\u0430 Lazarus'], 
     [13702, u'ru', u'\u041c\u0430\u0442\u0435\u043c\u0430\u0442\u0438\u0447\u0435\u0441\u043a\u0430\u044f \u044d\u043a\u043e\u043d\u043e\u043c\u0438\u043a\u0430']]
a

[[23126, 'en', 'Compass - powerful SASS library that makes your life easier'],
 [21617, 'en', 'Preparing for the AP* Computer Science A Exam — Part 2'],
 [16627, 'es', 'Aprende Excel: Nivel Intermedio by Alfonso Rinsche'],
 [11556,
  'es',
  'Aprendizaje Colaborativo by UNID Universidad Interamericana para el Desarrollo'],
 [16704, 'ru', 'Программирование на Lazarus'],
 [13702, 'ru', 'Математическая экономика']]

In [7]:
@f.udf(returnType=DoubleType())
def cos_sim(v,u):
    try:
        return float(v.dot(u))/float(v.norm(2)*u.norm(2))
    except:
        return 0

In [8]:
dic = dict()

for i in a:
    df2 = df.filter((df.id != i[0]) & (df.lang == i[1]))
    df3 = df.filter(df.id == i[0])[['features']]
    df4 = df2.join(df3.withColumnRenamed('features', 'temp'))
    df4 = df4.withColumn('cos_sim', cos_sim(df4.features, df4.temp))
    dic[i[0]] = list(df4.orderBy(df4.cos_sim.desc(), df4.name, df4.id).limit(10)[['id']].toPandas()['id'])
print(dic)

{23126: [14760, 13665, 13782, 20638, 24419, 15909, 2724, 25782, 17499, 13348], 21617: [21609, 21616, 21608, 22298, 21630, 21628, 21623, 21508, 21081, 19417], 16627: [11431, 11575, 12247, 17964, 5687, 17961, 16694, 12660, 25010, 5558], 11556: [16488, 468, 13461, 23357, 19330, 7833, 9289, 10447, 22710, 11340], 16704: [1236, 1247, 1365, 1273, 20288, 1164, 8186, 1233, 8203, 875], 13702: [864, 21079, 8313, 1041, 28074, 8300, 1033, 13057, 21025, 1111]}


In [9]:
with open('lab02.json', 'w') as f:
    json.dump(dic, f)

In [10]:
spark.stop()